In [11]:
import keras
import keras.backend as K
from keras import layers, Model, models
from keras.utils import plot_model
import numpy as np
from keras import callbacks
import tensorflow as tf
from sklearn.model_selection import train_test_split
# from sklearn.model_selection import cross_val_score

# Baseline model
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2)
x_train, x_test, x_val = x_train.astype('float32')/255, x_test.astype('float32')/255, x_val.astype('float32')/255
x_train, x_test, x_val = np.expand_dims(x_train, -1), np.expand_dims(x_test, -1), np.expand_dims(x_val, -1)

In [12]:
def build_model():
    model = models.Sequential([
        layers.Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1)),
        layers.MaxPooling2D((2,2)),
        layers.Dropout(0.3),
        layers.BatchNormalization(),
        layers.Conv2D(64, (3,3), activation='relu'),
        layers.MaxPooling2D((2,2)),
        layers.Dropout(0.3),
        layers.BatchNormalization(),
        layers.Conv2D(64, (3,3), activation='relu'),
        layers.Dropout(0.3),
        layers.BatchNormalization(),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(10, activation='softmax')
    ])
    return model


In [13]:
K.clear_session()
model = build_model()
plot_model(model, to_file="MinInception.png", show_shapes=True)
model.compile(loss='sparse_categorical_crossentropy', metrics=['acc'], optimizer='rmsprop')

In [15]:
reduce_lr = callbacks.ReduceLROnPlateau(
    monitor='val_acc', 
    min_lr=1e-6, 
    patience=3, 
    factor=0.4,
    min_delta=0.005,
    verbose=1
)
es = callbacks.EarlyStopping(
    min_delta=0.001,
    patience=5,
    verbose=1,
    restore_best_weights=True
)

tb = callbacks.TensorBoard(
    histogram_freq=2,
    write_graph=True,
    write_images=True
)

model.fit(
    x_train, y_train,
    epochs = 100,
    batch_size=128,
    validation_data=(x_val, y_val),
    callbacks=[reduce_lr, es, tb]
)


Epoch 1/100
375/375 [==============================] - 20s 34ms/step - loss: 0.5449 - acc: 0.8263 - val_loss: 1.8903 - val_acc: 0.4304
Epoch 2/100
375/375 [==============================] - 9s 23ms/step - loss: 0.0881 - acc: 0.9729 - val_loss: 0.0501 - val_acc: 0.9841
Epoch 3/100
375/375 [==============================] - 9s 24ms/step - loss: 0.0681 - acc: 0.9786 - val_loss: 0.0410 - val_acc: 0.9877
Epoch 4/100
375/375 [==============================] - 9s 25ms/step - loss: 0.0564 - acc: 0.9835 - val_loss: 0.0380 - val_acc: 0.9876
Epoch 5/100
375/375 [==============================] - 9s 23ms/step - loss: 0.0505 - acc: 0.9848 - val_loss: 0.0374 - val_acc: 0.9879

Epoch 00005: ReduceLROnPlateau reducing learning rate to 0.0004000000189989805.
Epoch 6/100
375/375 [==============================] - 9s 23ms/step - loss: 0.0423 - acc: 0.9867 - val_loss: 0.0280 - val_acc: 0.9909
Epoch 7/100
375/375 [==============================] - 9s 24ms/step - loss: 0.0330 - acc: 0.9900 - val_loss: 0.029